In [ ]:
!pip install huggingface_hub transformers bitsandbytes datasets

In [ ]:
!pip install -U transformers accelerate bitsandbytes torch torchvision torchaudio

## 모델 불러오기

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# 4비트 양자화
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# 모델명
model_name = 'google/gemma-3-4b-it'

# 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)

# LoRA를 모델에 적용
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### SFT 데이터

In [ ]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/LLM/planty/data/sft_dataset

/content/drive/MyDrive/LLM/planty/data


### 데이터 불러오기

In [ ]:
import json
from datasets import Dataset

#json 데이터 로드
with open('sft_dataset_cleaned.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

dataset = Dataset.from_dict({"question": [item["question"] for item in data], "answer": [item["answer"] for item in data]})

In [ ]:
dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 1242
})

In [ ]:
dataset['question'][:10]

['분갈이는 얼마에 한 번씩 하나요?',
 '분갈이에 가장 좋은 시기는?',
 '분갈이가 필요한 때를 어떻게 알 수 있나요?',
 '분갈이에 필요한 도구와 용기의 선택: 도구',
 '분갈이에 필요한 도구와 용기의 선택: 용기',
 '분갈이에 필요한 도구와 용기의 선택: 용기와 식물의 비율',
 '분갈이에 필요한 도구와 용기의 선택: 용기(화분)의 선택',
 '분갈이에 어떤 흙을 사용해야 하는가?',
 '분갈이할 때 산이나 주변에서 볼 수 있는 흙도 가능한가? 아니면, 꼭 구입한 토양만 써야 하는가?',
 '분갈이 시 흙의 양은?']

In [ ]:
dataset['answer'][:5]

['성장이 왕성하여 1년마다 사는 집이 비좁아지는 식물은 매년 봄에, 2년 정도 되어야새집이 필요한 식물은 2년마다, 성장이 아주 느린 식물은 필요한 시기에 해 준다.',
 '식물도 동물처럼 휴식이 필요하며 대부분의 식물은 한겨울을 휴식기로 선택한다. 그러나 봄이 오면 다시 새로운 생명 활동을 시작하므로 아직 꽃망울이 맺히기 전에 분갈이가 필요한 식물들은 분갈이를 한다. 그러나 너무 이른 봄보다는 좀 더 따뜻해졌을때(4~5월경) 새 집으로 이사하고 싶어 하는 식물들도 많다.',
 '분갈이가 필요할 때 식물들은 열심히 신호를 보낸다. 뿌리가 배수공을 빠져나와 소리를 지르는 광경은 흔히 볼 수 있는 풍경이다. 또한 물을 줄 때마다 이미 너무 커 버려 작은 화분이 자꾸 넘어지는 모습에서 우리는 식물의신호를 읽을 수 있다.',
 '새롭게 옮겨 심을 화분, 모종삽, 막대기, 물뿌리개, 배양토, 화분망 등',
 '식물에게 용기의 선택은 사람에게 살 집을 선택하는 것과 같다. 우리는 미관상 보기에만 좋은 집이 살기 편하다고 이야기하지는 않는다. 용기는 화분 안에 들어갈 흙의 온도와 수분의 유지, 조절에도 관여하며 식물을 더욱 아름답게, 편안하게 만드는 역할도 한다.']

### 토크나이징

In [ ]:
# 데이터 전처리
def tokenize_function(examples):
    prompt = [f"Q: {q}\nA: {a}" for q, a in zip(examples["question"], examples["answer"])]
    tokenized = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1242 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1242
})

## 파인튜닝

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = "./gemma_sft",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    save_strategy="epoch",
    learning_rate=2e-5,
    push_to_hub=False
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=468, training_loss=19.850289630074787, metrics={'train_runtime': 5364.5302, 'train_samples_per_second': 0.695, 'train_steps_per_second': 0.087, 'total_flos': 3.675671404373606e+16, 'train_loss': 19.850289630074787, 'epoch': 3.0})

## 모델 저장

In [ ]:
model.save_pretrained('./gemma_sft')
tokenizer.save_pretrained("./gemma_sft")

('./gemma_sft/tokenizer_config.json',
 './gemma_sft/special_tokens_map.json',
 './gemma_sft/tokenizer.model',
 './gemma_sft/added_tokens.json',
 './gemma_sft/tokenizer.json')

In [ ]:
from huggingface_hub import login, create_repo
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. Hugging Face 로그인
login(token='your_hf_token')

# 2. 저장소 생성 (저장소가 없다면 생성)
repo_id = "yerim00/gemma_sft"  # 원하는 repo_id 입력
create_repo(repo_id, private=False)  # 공개 저장소 생성 (비공개는 private=True)

# 3. 모델과 토크나이저 로드 (로컬 경로)
model_path = './gemma_sft'  # 로컬에 저장된 모델 경로
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 4. 모델과 토크나이저를 Hugging Face Hub에 업로드
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/8.93M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yerim00/gemma_sft/commit/e21e29190b3505694a1ab0647b29b731f56de453', commit_message='Upload tokenizer', commit_description='', oid='e21e29190b3505694a1ab0647b29b731f56de453', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yerim00/gemma_sft', endpoint='https://huggingface.co', repo_type='model', repo_id='yerim00/gemma_sft'), pr_revision=None, pr_num=None)

### 저장한 모델 불러오기

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = 'yerim00/gemma_sft'

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [ ]:
# 입력 프롬프트 설정
prompt = "다육식물은 겨울철에 어떻게 관리해야 하나요?"

# 입력을 토큰화
inputs = tokenizer(prompt, return_tensors="pt")

# 모델을 사용하여 텍스트 생성
output = model.generate(**inputs, max_length=200, do_sample=True, top_k=50, top_p=0.95)

# 생성된 텍스트 디코딩
plant = tokenizer.decode(output[0], skip_special_tokens=True)

print("번역:")
print(plant)

번역:
다육식물은 겨울철에 어떻게 관리해야 하나요?

겨울철에는 다육식물 관리가 쉽지 않지만, 몇 가지 주의 사항을 지키면 겨울철에도 건강하게 유지할 수 있습니다.

1. 시든 가지 제거하기: 겨울철에는 다육식물의 생장이 멈추므로 시든 가지를 제거하여 영양분 낭비를 막습니다.

2. 물 주기 줄이기: 겨울철에는 다육식물의 생장이 멈추므로 물 주는 횟수를 줄여야 합니다. 물을 줄 때는 과도한 물 공급을 피하고, 흙이 완전히 말랐을 때 물을 줍니다.

3. 온도 유지하기: 다육식물은 낮은 온도에서도 잘 자라지만, 너무 낮은 온도에는 피해야 합니다. 적정 온도는 10~20℃입니다.

4. 햇빛 충분히 받기: 겨울철에는 햇빛이


In [ ]:
import torch
import time

device = torch.device("cpu")

# 추론 최적화
def generate_answer(prompt, max_length=50):
    # 시간 측정 시작
    start_time = time.time()

    # 토큰화
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # 추론 시 그라디언트 계산 비활성화 (추론 성능 최적화)
    with torch.no_grad():
        # 텍스트 생성
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,  # 생성의 다양성 제어
            top_p=0.9,        # nucleus sampling
            top_k=50,         # 상위 k개 토큰 샘플링
            do_sample=True    # 샘플링을 통해 더 자연스러운 답변 생성
        )

    # 출력 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 시간 측정 종료
    end_time = time.time()

    # 소요된 시간 출력
    print(f"답변 생성 시간: {end_time - start_time:.4f} 초")

    return generated_text

# 예시 사용
prompt = "다육식물은 겨울철에 어떻게 관리해야 하나요?"
answer = generate_answer(prompt)
print("생성된 답변:", answer)


답변 생성 시간: 14.0695 초
생성된 답변: 다육식물은 겨울철에 어떻게 관리해야 하나요?

다육식물은 겨울철에 온도 변화와 햇빛 부족에 취약하므로, 적절한 관리가 필요합니다.

**1. 온도 관리


In [ ]:
import torch
import time
from transformers import AutoTokenizer, AutoModelForCausalLM

# 입력 프롬프트 설정
prompt = "다육식물은 겨울철에 어떻게 관리해야 하나요?"

# 시간 측정 시작
start_time = time.time()

# 입력을 토큰화
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 모델을 사용하여 텍스트 생성
output = model.generate(**inputs, max_length=200, do_sample=True, top_k=50, top_p=0.95)

# 생성된 텍스트 디코딩
plant = tokenizer.decode(output[0], skip_special_tokens=True)

# 시간 측정 종료
end_time = time.time()

# 소요된 시간 출력
print(f"답변 생성 시간: {end_time - start_time:.4f} 초")

# 생성된 답변 출력
print("번역:")
print(plant)

답변 생성 시간: 69.3196 초
번역:
다육식물은 겨울철에 어떻게 관리해야 하나요?
다육식물은 겨울철에 추위에 약하므로, 적절한 관리가 필요합니다. 
* **온도:** 겨울철에는 다육식물에 적절한 온도를 유지해야 합니다. 보통 5~10℃ 정도가 적당하며, 너무 낮은 온도는 식물의 성장을 저해할 수 있습니다.
* **햇빛:** 겨울철에는 햇빛이 부족하기 때문에, 식물을 햇빛이 잘 드는 곳으로 옮겨주는 것이 좋습니다. 햇빛이 부족하면 식물이 비대해지고, 개체 수가 줄어들 수 있습니다.
* **물주기:** 겨울철에는 식물의 생육 속도가 느려지기 때문에, 물주기를 줄여야 합니다. 보통 2~3주에 한 번 정도가 적당하며, 물을 줄 때는 흙이 완전히 마른 것을 확인한
